<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DQN

이번 프로젝트는 DQN 모델을 구현하고 훈련시켜봄으로써 DQN 아키텍처에 대해 깊이 이해하는 것을 목적으로 한다.

# 논문에서 제시하는 기본 접근방법들의 문제점과 해결법.

1. 큐러닝의 한계.

기존 큐러닝은 샘플링을 통해 상태-행동 쌍에 대한 큐값을 업데이트하는 방법을 사용한다. 하지만 이는 전혀 실용적이지 않은데 그 이유는 Q 값을 각 상태, 행동 쌍에 대해 따로따로 추정하고 일반화가 전혀 이루어지지 않기 때문에 상태, 행동 쌍이 거의 무한한 환경에서는 사용이 불가능하기 때문이다.
DQN에서는 이 문제를 해결하기위해 딥러닝 모델을 사용해 Q값을 일반화하고 추정하며 이를 Q-network라고 한다.

2. 딥러닝 모델을 강화학습에 적용할때 직면하는 어려움.

딥러닝모델을 강화학습에 이식하는것에는 여러 어려움이 있다. 딥러닝모델을 훈련시키기 위해서는 라벨링된 빅데이터가 필요한 반면 강화학습은 라벨링된 데이터가 없고 희박하거나 noisy하고 지연된 보상으로부터 학습해야한다. 또 데이터가 독립적이라고 가정하는 딥러닝과 달리 강화학습은 상관관계가 큰 sequence를 사용하고 강화학습의 데이터 분포는 고정된 딥러닝의 경우와다르게 행동정책이 업데이트됨에 따라 달라진다. DQN은 이를 극복하기위해 experience replay mechanism을 사용해 과거경험을 저장하고 여기서 무작위로 샘플링한 데이터로 모델을 업데이트하는 방법을 사용한다.

3. 기존 큐값추정 딥러닝모델의 계산비용 문제.

기존 아키텍쳐는 상태-행동쌍을 입력받아 큐값을 계산하는 방식을 사용했다. 하지만 이는 선택 가능한 행동이 많아질수록 계산비용히 선형적으로 증하게된다. 반면 DQN에서는 상태만을 입력으로 받고 가능한 행동의 수만큼 큐값을 출력하는 구조를 사용함으로써 이런 문제를 해결했다.

# DQN 이키텍쳐 특징

1. experience replay mechanism

DQN은 위에서 제시한 문제를 해결하기위해 experiemce replay mechanism을 사용한다. experience replay mechanism에 저장된 경험들은 무작위로 샘플링해 Q-network의 업데이트에 사용하며 이런 방법은 여러 장점을 갖는다. 첫번째 장점으로는 저장된 경험 데이터들이 잠재적으로 여러번 사용되 수 있으므로 좋은 데이터 효율을 갖는다는 것 이다. 두번째로는 강화학습의 특성상 연이은 경험 데이터들은 강한 상관관계를 갖고 있는데 저장된 경험들을 무작위로 샘플링 함으로써 이를 완화할 수 있다는 점이다. 마지막으로 많은 경험들로 부터 샘플링된 데이터는 경험들을 평균내어 학습을 부드럽게 하고 파라미터의 진동이나 발산을 방지할 수 있다는 점 이다. 하지만 단점으로 각 경험들의 중요성을 무시한다는 점이 있다. 중요한 경험이 있더라도 가중없이 무작위샘플린되며 메모리 용량이 다차면 역시 아무리 중요한 경험이라도 순서에따라 삭제된다.

2. Q러닝을 사용하는 이유.

만약 DQN을 on-policy 학습으로 구현한다면 타겟정책과 행동정책이 같으므로 현재 파라미터가 파라미터가 학습되는 다음 데이터 샘플을 결정하게 될 것 이다. 이렇게되면 학습샘플은 행동정책에따라 편향될것이고 이는 치명적인 문제를 야기할 가능성이 크다. DQN은 off-policy방법인 큐러닝을 사용함으로써 이런 문제를 방지한다.

3. 타겟네트워크

DQN에서 제시한 Loss function

$L_i(\theta_i) = E_{s, a-p(.)}[(y_i - Q(s,a;\theta_i))^2]$

$ y_i = E_{s'-\varepsilon}[r + \gamma max_{a'}Q(s', a';\theta_{i-1})] $

을 살펴보면 타겟을 만드는 타겟네트워크와 큐값을 추정하 큐네트워크의 파라미터 시점이 다르다는 것을 알 수 있다. 여기서 중요한 지점은 $ \theta_{i-1} $다. Q-network가 업데이되는동안 타겟네트워크 업데이트돼지않고 고정된 파라미터로 타겟값을 추정한다. 이는 학습에 사용될 데이터의 분포가 파라미터에 의존하기 때문이다. 이렇게 함으로써 타깃값이 너무 빠르게 변화하는것을 방지해 학습안정성을 높일 수 있다.



# 구현

In [ ]:
!pip install gymnasium
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]
!pip install imageio
!pip install imageio-ffmpeg

In [ ]:
import gymnasium as gym
import imageio
from gymnasium.wrappers import FrameStack, GrayScaleObservation, NormalizeObservation
import torch
from torch import nn
import numpy as np
import torch.optim as optim
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from google.colab import drive
from tqdm.auto import tqdm
import torchvision.transforms as transforms

drive.mount('/content/drive')

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 한 에피소드(목숨 1개)동안 greedy action으로 게임을 플레이한 후 에피소드 동안 얻은 총점을 반환.

def play_game(model, env, file_name, record : bool = False):

    total_reward = 0
    frames = []

    for i in range(10):

        obs, info = env.reset()
        if record == True:

            frames.append(env.render())

        obs, reward, terminated, truncated, info = env.step(1)
        if record == True:
            frames.append(env.render())

        while(terminated == False and truncated == False and info['lives'] == 5):

            _, action = model.greedy_action(torch.tensor(np.array(obs)).to(DEVICE))
            obs, reward, terminated, truncated, info = env.step(action)
            if record == True:
                frames.append(env.render())

            total_reward += reward

    # 동영상 저장.
    if record == True:
        with imageio.get_writer(f'/content/drive/MyDrive/동영상/{file_name}.mp4', fps=30, ) as video:
            for frame in frames:
                video.append_data(frame)

    total_reward = total_reward / 10

    return total_reward

def model_update(q_model, target_model, optimizer, buffer, batch_size):

    obs_batch, action_batch, reward_batch, nobs_batch = buffer.sample(batch_size)

    obs_batch = torch.tensor(np.array(obs_batch)).float().to(DEVICE)
    action_batch = torch.tensor(np.array(action_batch)).float().to(DEVICE)
    reward_batch = torch.tensor(np.array(reward_batch)).float().to(DEVICE)
    nobs_batch = torch.tensor(np.array(nobs_batch)).float().to(DEVICE)

    with torch.no_grad():
            y, _ = target_model.greedy_action(nobs_batch)
            y = 0.99*y # 할인계수 0.99
            y = y + reward_batch

    q = q_model.generate_q(obs_batch, action_batch)

    loss = F.mse_loss(y, q)

    optimizer.zero_grad()  # 그래디언트 초기화
    loss.backward()  # 역전파
    optimizer.step()  # 가중치 업데이트

    loss = loss.item()

    return loss

def step_and_stack(model, env, buffer, epsilon, step_size):

    obs, reward, terminated, truncated, info = env.step(1) # 스타트 버튼

    for i in (range(step_size)):

        if terminated == False and truncated == False and info['lives'] == 5:

            action = model.epsilon_greedy_action(torch.tensor(np.array(obs)).to(DEVICE), epsilon)
            nobs, reward, terminated, truncated, info = env.step(action)

            buffer.add([obs, action, reward, nobs])
            obs = nobs

        else:

            obs, info = env.reset()
            nobs, reward, terminated, truncated, info = env.step(1)
            buffer.add([obs, 1, reward, nobs])
            obs = nobs

    return

�
=
0
γ=0: 에이전트는 오직 즉각적인 보상만을 고려합니다. 즉, 장기적인 보상은 고려 대상에서 제외됩니다.
�
γ에 가까운 1: 에이전트는 미래 보상을 거의 현재 보상만큼 중요하게 고려합니다. 이는 장기적인 보상을 최대화하려는 전략에 해당합니다.
할인 계수 설정의 중요성
할인 계수는 강화학습 모델이 어떤 종류의 전략을 학습할지를 결정하는 핵심 요소 중 하나입니다. 너무 낮은 값은 에이전트가 단기적인 결과에만 초점을 맞추도록 하고, 너무 높은 값은 계산을 복잡하게 하고 학습을 불안정하게 만들 수 있습니다. 또한, 너무 높은 값은 에이전트가 먼 미래의 보상을 과도하게 평가하게 만들어 현재의 최적의 행동을 놓칠 수 있습니다.

일반적인 할인 계수 값
실제 사용에서는 할인 계수를 0.9 ~ 0.99 범위 내에서 설정하는 것이 일반적입니다. 특정 문제에 대한 최적의 값은 실험을 통해 결정되어야 하며, 문제의 특성과 에이전트가 학습해야 할 전략의 장기적인 성격에 따라 달라질 수 있습니다.

0.9는 비교적 짧은 시계열에서 미래 보상을 고려하는 경우에 적합할 수 있습니다.
0.99는 장기적인 보상을 더 중요하게 고려할 때 사용됩니다. 이는 에이전트가 더 장기적인 전략을 학습하게 하려는 경우에 유용합니다.


Experience replay

In [ ]:
class ExperienceReplayMemory:
    def __init__(self, capacity):
        self.capacity = capacity  # 메모리의 최대 저장 용량
        self.memory = []  # 경험을 저장할 리스트

    def add(self, experience):
        """메모리에 경험을 추가합니다. 메모리가 꽉 찼을 경우 가장 오래된 경험을 제거합니다."""
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory.pop(0)  # 가장 오래된 경험을 제거
            self.memory.append(experience)

    def sample(self, batch_size):
        """메모리에서 무작위로 경험을 샘플링합니다."""
        sample = random.sample(self.memory, batch_size)

        obs_batch = [obs for obs, _, _, _ in sample]
        action_batch = [action for _, action, _, _ in sample]
        reward_batch = [reward for _, _, reward, _ in sample]
        nobs_batch = [nobs for _, _, _, nobs in sample]

        return obs_batch, action_batch, reward_batch, nobs_batch


    def __len__(self):
        """메모리에 저장된 경험의 수를 반환합니다."""
        return len(self.memory)

In [ ]:
class DQN(nn.Module):
    def __init__(self):

        super().__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(4, 16, kernel_size = 8, stride = 4),
                                   nn.ReLU())

        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size = 4, stride = 2),
                                   nn.ReLU())

        self.fc = nn.Sequential(nn.Linear(2592, 256),
                                nn.ReLU(),
                                nn.Linear(256, 4))

    def forward(self, x):

        with torch.no_grad():
            x = self.preprocessing(x)

        x = self.conv1(x)
        x = self.conv2(x)
        x = torch.flatten(x,start_dim=1)
        x = self.fc(x)

        return x

    def greedy_action(self, x):

        with torch.no_grad():
            x = self.forward(x)
            x, idx = torch.max(x, 1)

        return x, idx

    def epsilon_greedy_action(self, x, epsilon):

        with torch.no_grad():
            if random.random() > epsilon:
                x, idx = self.greedy_action(x)
                x = idx.item()

            else:
                x = random.randrange(4)

        return x

    def generate_q(self, x, action_batch):

        x = self.forward(x)
        action_batch = action_batch.unsqueeze(1).long()
        q = torch.gather(x, 1, action_batch)
        q = q.squeeze(1)

        return q

    def preprocessing(self, x):

        # x = np.array(x)
        # x = torch.tensor(x)

        if x.ndim == 3:
            x = x.unsqueeze(0)

        x = x[:, :, 34:-16, :]
        resize_transform = transforms.Resize((84, 84))
        x = resize_transform(x)
        x = x.float() / 255.

        return x

In [ ]:
q_model = DQN().to(DEVICE)
t_model = DQN().to(DEVICE)
t_model.load_state_dict(q_model.state_dict())

In [ ]:
#환경 초기화
env = gym.make("ALE/Breakout-v5", render_mode='rgb_array', obs_type='grayscale')
env = FrameStack(env, 4)
obs, info = env.reset()

In [ ]:
# 매개변수 초기화.
epsilon = 1
buffer = ExperienceReplayMemory(1000000)
#optimizer = optim.Adam(q_model.parameters(), lr= 0.00025)
optimizer = optim.RMSprop(q_model.parameters(), lr=0.00025, alpha=0.95, eps=1e-6)
reward_r = []
loss_r = []
epsilon_r = []

In [ ]:
#step_and_stack(q_model, env, buffer, epsilon, step_size = 50000)

for i in tqdm(range(900000)):

    i = i+100000

    step_and_stack(q_model, env, buffer, epsilon, step_size = 10)

    loss =
 model_update(q_model, t_model, optimizer, buffer, 64)
    loss_r.append(loss)

    if i % 5000 == 0:

        t_model.load_state_dict(q_model.state_dict())
        torch.save(q_model, f'/content/drive/MyDrive/모델/{str(i)}.pth')

        r = play_game(q_model, env, str(i), record = True)

        print(f"total reward : {r}")
        reward_r.append(r)

    if i % 500 == 0:
        epsilon = max(epsilon - 0.00045, 0.1)
        epsilon_r.append(epsilon)

In [ ]:

# reward_r 리스트를 reward.txt 파일에 저장
with open('/content/drive/MyDrive/reward.txt', 'w', encoding='utf-8') as file:
    for item in reward_r:
        file.write("%s\n" % item)

# loss_r 리스트를 loss.txt 파일에 저장
with open('/content/drive/MyDrive/loss.txt', 'w', encoding='utf-8') as file:
    for item in loss_r:
        file.write("%s\n" % item)

# epsilon_r 리스트를 epsilon.txt 파일에 저장
with open('/content/drive/MyDrive/epsilon.txt', 'w', encoding='utf-8') as file:
    for item in epsilon_r:
        file.write("%s\n" % item)


# 동작확인용.

# def step_and_stack(model, env, epsilon, step_size):

#     frames = []

#     obs, reward, terminated, truncated, info = env.step(1) # 스타트 버튼

#     frames.append(env.render())

#     for i in (range(step_size)):

#         if terminated == False and truncated == False and info['lives'] == 5:

#             action = model.epsilon_greedy_action(obs, epsilon)
#             nobs, reward, terminated, truncated, info = env.step(action)

#             frames.append(env.render())

#             buffer.add([obs, action, reward, nobs])
#             obs = nobs

#         else:

#             obs, info = env.reset()
#             nobs, reward, terminated, truncated, info = env.step(1)

#             frames.append(env.render())

#             buffer.add([obs, action, reward, nobs])
#             obs = nobs

#     with imageio.get_writer('/content/drive/MyDrive/동영상/my_game_test.mp4', fps=30) as video:
#         for frame in frames:
#             video.append_data(frame)